In [ ]:
from pathlib import Path
from cvla.hf_model_class import cVLA_wrapped

model_location = Path("/home/houman/cVLA_test/models")
#model_path = model_location / "clevr-act-7-depth_l40" / "checkpoint-4687"
#model_path = model_location / "clevr-act-6-var-cam2_hf_af_lr3e5" / "checkpoint-4687"
#model_path = model_location / "clevr-act-7-depth_rbg20" / "checkpoint-4687"
model_path = model_location / "mix30obj_text_debug" / "checkpoint-4687"
#model_path = model_location / "clevr-act-7-depth_rbg60" / "checkpoint-4687"

model_inst = cVLA_wrapped(model_path=model_path)

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import time
import subprocess
import numpy as np
from PIL import Image   
from matplotlib import pyplot as plt
from IPython.display import display, HTML

from cvla.utils_trajectory import DummyCamera
from cvla.utils_vis import render_example

def get_new_image(image_dir = Path("/home/houman/cVLA_test/saved_images/dataset"), image_number=1):
    process = subprocess.Popen("python /home/houman/catkin_ws/src/franka_utils/scripts/get_image.py {}".format(image_number),shell=True)
    time.sleep(1)
    depth_image = np.load(image_dir / f"depth_image_{image_number}.npy")
    rgb_image = Image.open( image_dir / f"rgb_image_{image_number}.png")
    if process.poll() is None:
        process.kill()
        print("Process killed after 1 second.")
    else:
        print("Process finished before timeout.")


    depth_image = np.load(image_dir / f"depth_image_{image_number}.npy")
    rgb_image = Image.open( image_dir / f"rgb_image_{image_number}.png")
    return rgb_image, depth_image

In [ ]:
# Get a new image
rgb_image, depth_image = get_new_image()



In [ ]:
import torch
from torchvision.transforms import v2

# Generate the prefix
x = "black sponge"
y = "plate"
action_text = "put the {} inside the {}".format(x, y)

crop = True
center_crop = v2.CenterCrop(360)

image_width, image_height = rgb_image.size
camera_extrinsic = [[[1, 0, 0.0, 0.0], [0, 1, 0, 0], [0, 0, 1, 0]]]
camera_intrinsic = [[[260.78692626953125, 0.0, 322.3820495605469],[ 0.0, 260.78692626953125, 180.76370239257812],[0.0, 0.0, 1.0]]]
camera = DummyCamera(camera_intrinsic, camera_extrinsic, width=image_width, height=image_height)

base_to_tcp_pos = torch.tensor([[[-0.7487, -0.3278,  0.7750]]])
base_to_tcp_orn = torch.tensor([[[ 1,  0, 0, 0]]])  # quaternion w, x, y, z 

_, _, robot_state = model_inst.enc_model.encode_trajectory(base_to_tcp_pos, base_to_tcp_orn, camera)

if model_inst.return_depth:
    images = [rgb_image, depth_image]
    print(images[0].shape, images[0].dtype)  # (720, 1280, 3) uint8 depth image encoded
    print(images[1])  # <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x720 at 0x715AEF7667B0>
else:
    images = rgb_image
    if crop:
        images = center_crop(images)


pred_text = model_inst.predict(images, action_text, robot_state)

In [ ]:
decc_model = model_inst.enc_model.decode_caption
sample = dict(suffix=pred_text[0], prefix=action_text + " " + robot_state)

image_crop_width, image_crop_height = images.size
camera_after_crop = DummyCamera(camera_intrinsic, camera_extrinsic, width=image_crop_width, height=image_crop_height)

print(sample["prefix"])
print("pred_text", pred_text)
html_imgs = ""
html_imgs += render_example(images, label=sample["suffix"], text=sample["prefix"], camera=camera_after_crop, enc=model_inst.enc_model)    
display(HTML(html_imgs))

In [ ]:
import numpy as np

def find_closest_valid_pixel(depth_image, target_row, target_col):
    """
    Find the closest valid pixel value to the target position in a depth image.
    
    Args:
        depth_image: 2D numpy array representing the depth image
        target_row: Row index of the position to find closest valid pixel for
        target_col: Column index of the position to find closest valid pixel for
    
    Returns:
        float: Value of the closest valid pixel, or None if no valid pixels exist
    """
    rows, cols = depth_image.shape
    print (rows, cols)
    if target_row >= rows or target_col >= cols:
        raise ValueError("Target coordinates exloc0476ceed image dimensions")
    closest_pixel_coords = None
    # Get coordinates of all valid pixels
    valid_coords = [(i,j) for i in range(rows) 
                    for j in range(cols) 
                    if not np.isnan(depth_image[i,j])]
    
    # If target posienc_modeltion already has a valid value, return it
    if not np.isnan(depth_image[target_row, target_col]):
        print("The coordinate already has a Non-Nan value!")
        return depth_image[target_row, target_col], [target_row, target_col]
    
    # If no valid pixels exist, return None
    if not valid_coords:
        return None, None
    
    # Find closest valid pixel using Manhattan distance
    min_dist = float('inf')
    closest_value = None
    
    for row, col in valid_coords:
        dist = abs(target_row - row) + abs(target_col - col)
        if dist < min_dist:
            min_dist = dist
            closest_value = depth_image[row, col]
            closest_pixel_coords = [row,col]
            
    return closest_value, closest_pixel_coords

# Example usage

# Test finding closest value for a NaN position


In [ ]:

print(pred_text)
traj_c = model_inst.enc_model.decode_caption(pred_text[0], camera=camera)




import torch
from copy import deepcopy
traj_c_fixed = deepcopy(traj_c)
#traj_c_fixed[0]

#color_value = np.asanyarray(rgb_image)[point[1], point[0]]
points = []
for point_idx in (0,1):
    point = np.array(traj_c[0][point_idx,:2].round().numpy(), dtype=int)
    row, col = point[1], point[0]  # Position containing NaN
    closest_value, closest_pixel_coords = find_closest_valid_pixel(depth_image, row, col)
    print(f"Closest valid value to position ({row}, {col}): {closest_value} at ({closest_pixel_coords[0]}, {closest_pixel_coords[1]})")
    depth_value = closest_value #depth_image[point[1], point[0]]

    print("point", point, depth_value)
    points.append(point)

    # if point_idx == 0:  # TODO(houman): find some good values, higher=further
    #     depth_value += 0.02
    # elif point_idx == 1:
    #     depth_value -= 0.03
    # else:
    #     raise ValueError

    print("depth_value", depth_value)
    traj_c_fixed[0][point_idx,2] = float(depth_value)

print("old", traj_c[0])
print("fixed", traj_c_fixed[0])
print()

from scipy.spatial.transform import Rotation as R
from mani_skill.utils.structs import Pose
from mani_skill.examples.utils_trajectory import unproject_points

curve_25d, quat_c = traj_c_fixed
# from camera to world coordinates
extrinsic_orn = R.from_matrix(camera.get_extrinsic_matrix()[:, :3, :3])
extrinsic = Pose.create_from_pq(p=camera.get_extrinsic_matrix()[:, :3, 3],
                                q=extrinsic_orn.as_quat(scalar_first=True))
quat_w = extrinsic.inv() * Pose.create_from_pq(q=quat_c)
curve_w = unproject_points(camera, curve_25d) 

curve_w, quat_w.get_q().unsqueeze(0)  # shape (P, 3 = u, v, d)    

print("done.")
print(curve_w)
print(quat_w)
    
print (np.shape(quat_w))
print (np.shape(quat_w.raw_pose.numpy()))
print (points)

In [ ]:
import numpy as np
from PIL import Image
from pathlib import Path

import matplotlib.pyplot as plt


#point = (360, 170) #width, height yellow point works
#point = (355, 170) #width, height yellow point fails
print (points[0][0])
point_1 = (points[0][0], points[0][1]) #width, height
point_2 = (points[1][0], points[1][1]) #width, height

fig, axs = plt.subplots(1,2)
axs[0].imshow(rgb_image)
axs[0].plot(point_1[0], point_1[1],'.-', color='lime')
axs[0].plot(point_2[0], point_2[1],'.-', color='red')

axs[1].imshow(depth_image)
axs[1].plot(point_1[0], point_1[1],'.-', color='lime')
axs[1].plot(point_2[0], point_2[1],'.-', color='red')

# color_value = np.asanyarray(rgb_image)[point[1], point[0]]
# depth_value = depth_image[point_2[1], point[0]]
# print (depth_image.shape)

# if depth_value is np.nan:
#     sample_range = # get dxd pixels centered at point
#     #get all depth_values
#     # get max of depth value
print("XXX", depth_value)



## Notes

1. Maybe sample depth range by default (?)
2. Remeber to do grasp point depth + 2.5 cm(?), placement depth - 5/10cm(?)
3. 

In [ ]:
Option 1:
(x,y,z) positions -> encode -> string -> 